In [ ]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
Query='''






DECLARE
  start_dt_YTD_snpsht DATE DEFAULT '2023-01-01';
DECLARE
  end_dt_YTD_snpsht DATE DEFAULT '2023-03-31';
DECLARE
  snpsht_start_mnth DATE DEFAULT '2023-03-01';



with order_data as (select *,PARSE_DATE('%d%h%Y',SUBSTR(CREATED_DT, 0,9)) as Created_date,  PARSE_DATE('%d%h%Y',SUBSTR(ACTIVATION_DT, 0,9)) as Activation_date
from `divgpras-pr-579355.SHS.Order_details_Jan2023_Mar2023`
)


, order_data_1 as (

select * , cast (CUST_ID as STRING) as Telus_Cust_ID,

CASE
      WHEN DATE_DIFF(DATE(Activation_date),Created_date, DAY) > 30 THEN 'Activated_30days+'
    ELSE
    'Activated_within_30days'
  END
    AS Day_gap

from order_data

)

,Vol_churn as (

select distinct BILLING_ACCOUNT_NUM as BAN,1 as Vol_churn ,PARSE_DATE('%d%h%Y',SUBSTR(ACTIVITY_DATE, 0,9)) as Vol_churn_date from `divgpras-pr-579355.SHS.VolChurn_Jan2023_Mar2023`

)

,InVol_churn as (

select distinct BILLING_ACCOUNT_NUM as BAN, 1 as InVol_churn,PARSE_DATE('%d%h%Y',SUBSTR(DLY_PROD_INSTNC_ACTVY_TS, 0,9)) as InVol_churn_date from `divgpras-pr-579355.SHS.InVolChurn_Jan2023_Mar2023`

)

,ADC_customer_details as (

select customer_id as adc_customer_id,dealer_customer_id  as Telus_customer_ID, date(join_date) as join_date

from `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details`
where dealer_name='TELUS Communications Inc.' 
and date(last_updt_ts)>=start_dt_YTD_snpsht and date(last_updt_ts)<= end_dt_YTD_snpsht
and date(join_date) >= start_dt_YTD_snpsht
QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY last_updt_ts DESC) = 1

)


,ADC_login_YTD as (

with date_sql as (
select  count(distinct date(login_dt_utc)) as count_of_dates from `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_daily_logins`
where date(login_dt_utc)<= end_dt_YTD_snpsht and date(login_dt_utc)>= start_dt_YTD_snpsht

)

select customer_id,sum(login_count) as sum_login_count,count (distinct date(login_dt_utc)) as number_of_login_days_YTD, (select count_of_dates from date_sql ) as count_of_dates_login_YTD
from `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_daily_logins`
where date(login_dt_utc)<= end_dt_YTD_snpsht and date(login_dt_utc)>= start_dt_YTD_snpsht
group by customer_id



)

,ADC_merge_data as (

select *
from ADC_customer_details a
left join ADC_login_YTD b
on a.adc_customer_id=b.customer_id


)

,ADC_final_data as (

select Telus_customer_ID, avg(number_of_login_days_YTD) as Avg_no_login_days_YTD
from ADC_merge_data 
group by Telus_customer_ID

)


select a.*,b.Vol_churn, b.Vol_churn_date,c.InVol_churn,c.InVol_churn_date,d.Telus_customer_ID,d.Avg_no_login_days_YTD,DATE_DIFF(end_dt_YTD_snpsht,a.Activation_date, DAY)  as Days_potential_login
-- round(d.number_of_login_days_YTD)*100/(DATE_DIFF(end_dt_YTD_snpsht,a.Activation_date, DAY)) as login_consistency_YTD



from order_data_1 a
left join Vol_churn b
on a.BILL_ACCOUNT_NUMBER=b.BAN
left join InVol_churn c
on a.BILL_ACCOUNT_NUMBER=c.BAN
left join ADC_final_data d
on a.Telus_Cust_ID=d.Telus_customer_ID
order by a.Created_date




'''

In [ ]:
DF=extract_bq_data(bq_client, sql=Query)

In [ ]:
DF.info()

In [ ]:
DF.head()

In [ ]:
DF.tail()

In [ ]:
# DF['adc_customer_id'].isna().sum()

In [ ]:
DF['Avg_no_login_days_YTD'].isna().sum()

In [ ]:
# DF['number_of_login_days_YTD']=np.where(DF['adc_customer_id'].isna(),0,DF['number_of_login_days_YTD'])

In [ ]:
DF.to_csv('Telus_Sales_Quality_report.csv',index=False)